# 003 Deeplearning

## Pablo Sánchez
## Braulio Marin 

In [ ]:
import pandas as pd 
import numpy as np 
import ta 
import matplotlib.pyplot as plt 
import optuna


In [ ]:
data = pd.read_csv("/Users/Usuario/Documents/10 SEMESTRE/Trading/Empezando_redes_neuronales/aapl_5m_train.csv").dropna()
data

### RSI, BOLLINGER, MACD

In [ ]:
import ta.momentum
import ta.volatility


rsi = ta.momentum.RSIIndicator(data.Close, window=67)
bb = ta.volatility.BollingerBands(data.Close, window=75, window_dev=1)
macd = ta.trend.MACD(data.Close, window_slow=20,window_fast=11, window_sign=21)


In [ ]:
dataset = data.copy()
dataset["RSI"] = rsi.rsi()
dataset["BB"] = bb.bollinger_mavg()
#MACD
dataset['macd_signal'] = macd.macd_signal()
dataset["MACD"] = macd.macd()

# Estas señales se tienen que optimizar con el proyecto de 1 y 2 personas 
dataset["RSI_BUY"] = dataset["RSI"] < 21 
dataset["RSI_SELL"] = dataset["RSI"] > 80 


dataset["BB_BUY"] = bb.bollinger_lband_indicator().astype(bool)
dataset["BB_SELL"] = bb.bollinger_hband_indicator().astype(bool)

dataset["MACD_BUY"] = False
dataset["MACD_SELL"] = False

macd = dataset["MACD"]
macd_signal = dataset["macd_signal"]

for i in range(1, len(dataset)):
    # Cruce alcista: señal de compra
    if macd.iloc[i] > macd_signal.iloc[i] and macd.iloc[i - 1] <= macd_signal.iloc[i - 1]:
        dataset.at[i, "MACD_BUY"] = True
    # Cruce bajista: señal de venta
    elif macd.iloc[i] < macd_signal.iloc[i] and macd.iloc[i - 1] >= macd_signal.iloc[i - 1]:
        dataset.at[i, "MACD_SELL"] = True

dataset = dataset.dropna()
dataset.head()






In [ ]:
capital = 1_000_000
com = 0.125 / 100

portfolio_value = [capital]

stop_loss = 0.16113
take_profit = 0.072
n_shares = 1000

wins = 0
losses = 0

active_long_positions = None
active_short_positions = None



for i, row in dataset.iterrows():
    # Close long positions
    if active_long_positions:
        # Closed by stop loss
        if row.Close < active_long_positions["stop_loss"]:
            pnl = row.Close * n_shares * (1 - com)
            capital += pnl
            active_long_positions = None

    if active_long_positions:
        # Closed by take profit
        if row.Close > active_long_positions["take_profit"]:
            pnl = row.Close * n_shares * (1 - com)
            capital += pnl
            active_long_positions = None

    # Close short positions
    if active_short_positions:
        # Closed by stop loss
        if row.Close > active_short_positions["stop_loss"]:
            entrada = active_short_positions["opened_at"]
            salida = row.Close
            pnl = (entrada - salida) * n_shares * (1 - com)
            capital += pnl
            active_short_positions = None

    if active_short_positions:
        # Closed by take profit
        if row.Close < active_short_positions["take_profit"]:
            entrada = active_short_positions["opened_at"]
            salida = row.Close
            pnl = (entrada - salida) * n_shares * (1 - com)
            capital += pnl
            active_short_positions = None

    # Open Long positions
    if (row.RSI_BUY + row.BB_BUY + row.MACD_BUY) >= 2 and active_long_positions is None:
        cost = row.Close * n_shares * (1 + com)
        if capital > cost:
            capital -= cost
            active_long_positions = {
                "datetime": row.Datetime,
                "opened_at": row.Close,
                "take_profit": row.Close * (1 + take_profit),
                "stop_loss": row.Close * (1 - stop_loss)
            }

    # Open short positions
    if (row.RSI_SELL + row.BB_SELL + row.MACD_SELL) >= 2 and active_short_positions is None:
        cost = row.Close * n_shares * com
        if capital > cost:
            capital -= cost
            active_short_positions = {
                "datetime": row.Datetime,
                "opened_at": row.Close,
                "take_profit": row.Close * (1 - take_profit),
                "stop_loss": row.Close * (1 + stop_loss)
            }


    # Calculate long positions value
    long_value = 0
    if active_long_positions:
        long_value = row.Close * n_shares

    # Calculate short positions value
    short_value = 0
    if active_short_positions:
        entrada = active_short_positions["opened_at"]
        short_value = (entrada - row.Close) * n_shares

        # Calculate portafolio value

    # Add portfolio value
    portfolio_value.append(capital + long_value + short_value)

portfolio_value[-1]

In [ ]:
fig, ax = plt. subplots (1, 1, figsize= (12, 6))
ax. plot (portfolio_value, label="Portfolio Value")
ax. legend ()
ax2 = ax. twinx ()
ax2. plot (data. Close, c="C1")


plt. show()

In [ ]:
import ta

def objective_func(trial, data,return_full=False):
    # Parámetros RSI
    rsi_window = trial.suggest_int("rsi_window", 10, 100)
    rsi_lower = trial.suggest_int("rsi_lower", 5, 35)
    rsi_upper = trial.suggest_int("rsi_upper", 65, 95)

    # Bollinger Bands
    bb_window = trial.suggest_int("bb_window", 10, 100)
    bb_window_dev = trial.suggest_int("bb_window_dev", 1, 3)

    # MACD
    slow_window = trial.suggest_int("window_slow", 13, 30)
    fast_window = trial.suggest_int("window_fast", 1, 12)
    window_signal = trial.suggest_int("window_sign", 1, 50)

    # Stop loss, take profit, cantidad de acciones
    stop_loss = trial.suggest_float("stop_loss", 0.01, 0.2)
    take_profit = trial.suggest_float("take_profit", 0.01, 0.2)
    n_shares = trial.suggest_categorical("n_shares", [1000, 1200, 3000, 5000,7000, 8000])

    # Dataset y cálculos de indicadores
    dataset = data.copy()
    dataset["RSI"] = ta.momentum.RSIIndicator(dataset.Close, window=rsi_window).rsi()
    bb = ta.volatility.BollingerBands(dataset.Close, window=bb_window, window_dev=bb_window_dev)
    dataset["BB_MAVG"] = bb.bollinger_mavg()
    dataset["BB_BUY"] = bb.bollinger_lband_indicator().astype(bool)
    dataset["BB_SELL"] = bb.bollinger_hband_indicator().astype(bool)

    macd = ta.trend.MACD(data.Close, window_slow=slow_window, window_fast=fast_window, window_sign=window_signal)
    dataset["MACD"] = macd.macd()
    dataset["MACD_SIGNAL"] = macd.macd_signal()
    dataset["MACD_BUY"] = False
    dataset["MACD_SELL"] = False

    # Señales RSI
    dataset["RSI_BUY"] = dataset["RSI"] < rsi_lower
    dataset["RSI_SELL"] = dataset["RSI"] > rsi_upper

    # Señales MACD (cruce)
    for i in range(1, len(dataset)):
        if dataset["MACD"].iloc[i] > dataset["MACD_SIGNAL"].iloc[i] and dataset["MACD"].iloc[i - 1] <= dataset["MACD_SIGNAL"].iloc[i - 1]:
            dataset.at[dataset.index[i], "MACD_BUY"] = True
        elif dataset["MACD"].iloc[i] < dataset["MACD_SIGNAL"].iloc[i] and dataset["MACD"].iloc[i - 1] >= dataset["MACD_SIGNAL"].iloc[i - 1]:
            dataset.at[dataset.index[i], "MACD_SELL"] = True

    dataset = dataset.dropna()

    capital = 1_000_000
    com = 0.00125
    portfolio_value = [capital]
    portfolio_dates = [dataset.index[0]]
    wins = 0
    losses = 0

    active_long_positions = None
    active_short_positions = None

    for i, row in dataset.iterrows():
        price = row.Close

        # Cierre de posición larga
        if active_long_positions:
            entry = active_long_positions["opened_at"]
            if price <= active_long_positions["stop_loss"] or price >= active_long_positions["take_profit"]:
                pnl = price * n_shares * (1 - com)
                capital += pnl
                if price > entry:
                    wins += 1
                else:
                    losses += 1
                active_long_positions = None

        # Cierre de posición corta
        if active_short_positions:
            entry = active_short_positions["opened_at"]
            if price >= active_short_positions["stop_loss"] or price <= active_short_positions["take_profit"]:
                pnl = (entry - price) * n_shares * (com)
                capital += pnl
                if price < entry:
                    wins += 1
                else:
                    losses += 1
                active_short_positions = None

        # Apertura de posición larga
        if (row.RSI_BUY + row.BB_BUY + row.MACD_BUY) >= 2 and not active_long_positions and not active_short_positions:
            cost = price * n_shares * (1 + com)
            if capital >= cost:
                capital -= cost
                active_long_positions = {
                    "datetime": row.Datetime,
                    "opened_at": price,
                    "take_profit": price * (1 + take_profit),
                    "stop_loss": price * (1 - stop_loss)
                }

        # Apertura de posición corta
        if (row.RSI_SELL + row.BB_SELL + row.MACD_SELL) >= 2 and not active_short_positions and not active_long_positions:
            margin = price * n_shares * com
            if capital >= margin:
                capital -= margin
                active_short_positions = {
                    "datetime": row.Datetime,
                    "opened_at": price,
                    "take_profit": price * (1 - take_profit),
                    "stop_loss": price * (1 + stop_loss)
                }

        # Valor actual del portafolio
        long_value = row.Close * n_shares if active_long_positions else 0
        short_value = (active_short_positions["opened_at"] - price) * n_shares if active_short_positions else 0
        portfolio_value.append(capital + long_value + short_value)
        portfolio_dates.append(row.name)
        
    if return_full:
        return {
            "portfolio": portfolio_value,
            "wins": wins,
            "losses": losses,
            "prices": dataset.Close
        }

    
    return portfolio_value[-1]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda x : objective_func(x,data), n_trials=100)



In [ ]:
study.best_value

In [ ]:
best_params = study.best_params
best_params

In [ ]:
# Evaluar con output completo
result = objective_func(optuna.trial.FixedTrial(best_params), data, return_full=True)
result

In [ ]:
import matplotlib.pyplot as plt

def plot_strategy(data, portfolio):
    fig, ax = plt.subplots(1, 1, figsize=(14, 6))
    
    # Graficar el valor del portafolio
    ax.plot(portfolio, label="Portfolio Value", color="C0", linewidth=2)
    ax.set_ylabel("Valor del Portafolio", color="C0")
    ax.tick_params(axis='y', labelcolor="C0")
    
    # Graficar el precio del activo en el eje derecho
    ax2 = ax.twinx()
    ax2.plot(data.Close, label="Precio del Activo", color="C1", linestyle="--")
    ax2.set_ylabel("Precio del Activo", color="C1")
    ax2.tick_params(axis='y', labelcolor="C1")
    
    # Títulos y leyenda
    plt.title("Estrategia de Trading: Portafolio vs Precio del Activo")
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np

def calculate_metrics(portfolio, prices, wins, losses):
    returns = np.diff(portfolio) / portfolio[:-1]
    if len(returns) == 0:
        return {
            "Sharpe Ratio": 0,
            "Sortino Ratio": 0,
            "Calmar Ratio": 0,
            "Win Rate": 0,
        }

    # Tasa libre de riesgo diaria (10% anual)
    risk_free_rate = 0.10 / 19656
    excess_returns = returns - risk_free_rate

    # Sharpe ratio
    sharpe = (
        np.mean(excess_returns) / np.std(returns) * np.sqrt(19656)
        if np.std(returns) > 0 else 0
    )

    # Sortino ratio
    downside = excess_returns[excess_returns < 0]
    sortino = (
        np.mean(excess_returns) / np.std(downside) * np.sqrt(19656)
        if len(downside) > 0 and np.std(downside) > 0 else 0
    )

    # Calmar ratio
    cumulative_return = portfolio[-1] / portfolio[0] - 1
    drawdown = portfolio / np.maximum.accumulate(portfolio) - 1
    max_drawdown = np.min(drawdown)
    calmar = (cumulative_return) / abs(max_drawdown) if max_drawdown < 0 else 0

    # Win rate
    total_trades = wins + losses
    win_rate = wins / total_trades if total_trades > 0 else 0

    return {
        "Sharpe Ratio": sharpe,
        "Sortino Ratio": sortino,
        "Calmar Ratio": calmar,
        "Win Rate": win_rate
    }

In [ ]:
# Calcular métricas
metrics = calculate_metrics(result["portfolio"], result["prices"], result["wins"], result["losses"])
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Graficar
plot_strategy(dataset,result['portfolio'])